# Chapter 5: The Dirty COW vulnerability in Rust

## Introduction
[**The Dirty COW vulnerability**](https://dirtycow.ninja/) represents a fascinating instance of a [**race condition**](https://en.wikipedia.org/wiki/Race_condition) [**vulnerability**](https://en.wikipedia.org/wiki/Vulnerability_(computing)) within the [**Linux kernel**](https://en.wikipedia.org/wiki/Linux_kernel). This flaw has been present in the kernel since **September 2007** but only came to light when [**it was discovered**](https://access.redhat.com/security/cve/CVE-2016-5195) and [**patched by the lord himself**](https://git.kernel.org/pub/scm/linux/kernel/git/torvalds/linux.git/commit/?id=19be0eaffa3ac7d8eb6784ad9bdbc7d67ed8e619) in **October 2016**. What sets this vulnerability apart is its wide-reaching impact, affecting all Linux-based operating systems, including the popular Android platform. The gravity of the situation lies in the potential consequences, as attackers exploiting this vulnerability can acquire root privileges, granting them god-tier control over the compromised system.

At its core, the vulnerability is embedded in the [**copy-on-write**](https://en.wikipedia.org/wiki/Copy-on-write) mechanism within the Linux kernel's code. The exploit allows attackers to manipulate protected files, even those designated as exclusively readable by them. This chapter delves into the complexities of the attack, dissecting its mechanisms and demonstrating how bad actors can leverage it to modify critical system files. A noteworthy example is the manipulation of the **`/etc/password`** file, showcasing how attackers can exploit the Dirty COW vulnerability to elevate their privileges to the root level, effectively taking over the entire system.

To fully comprehend the Dirty COW race condition vulnerability, it is crucial to explore its historical context. This flaw went undetected for almost a decade, highlighting the sneaky nature of certain security threats. The vulnerability was brought to light through meticulous research and analysis, emphasizing the perpetual need for careful investigation in the world of cybersecurity. Moreover, its discovery underscores the challenges inherent in maintaining the security of open-source systems, where complex codebases can port vulnerabilities over extended periods.

In terms of practical implications, the Dirty COW vulnerability has prompted widespread concern within the cybersecurity community. Security experts and Linux system administrators must remain careful, promptly patching affected systems and implementing robust security measures. The incident also serves as a stark reminder of the ever-evolving nature of cyber threats and the necessity for proactive defense mechanisms.

## Table of Contents

* [**Memory Mapping**](#1.-Memory-Mapping)
    * [**TODO**](#1.1-TODO)
       * [**TODO**](#1.1.1-TODO)

## 1. Memory Mapping

Kicking off the journey to comprehend the complexities of the **Dirty COW** vulnerability necessitates a solid grasp of the foundational concept of memory mapping through the use of the [**`libc::mmap`**](https://docs.rs/libc/latest/libc/fn.mmap.html) method. Within the Unix operating system, [**`mmap`**](https://man7.org/linux/man-pages/man2/mmap.2.html) empowers the seamless integration of files or devices into a process's memory space. This mechanism plays a crucial role in shaping how data is accessed and manipulated within a computer system.

By default, **`mmap`** employs [**file-backed mapping**](https://en.wikipedia.org/wiki/Mmap#File-backed_and_anonymous), establishing a [**symbiotic relationship**](https://en.wikipedia.org/wiki/Symbiosis) between an allocated portion of a process's virtual memory and corresponding files. When information is read from the mapped area, it dynamically translates into the retrieval of data from the associated file. This natural connection between memory and file operations forms the backbone of **`mmap`**'s functionality.

To shed light on this process, let's turn our attention to the following code snippet. This code snippet encapsulates the essence of memory mapping, showcasing how **`mmap`** is employed to create a link between a file and a process's memory space. This practical example offers valuable insights into the mechanics of memory mapping, serving as a guide for understanding the subsequent exploration of the Dirty COW vulnerability.

In [2]:
:dep libc = { version = "0.2.151" }

In [4]:
use libc::{
    MAP_FAILED, MAP_SHARED, PROT_READ, PROT_WRITE,
};
use std::fs::OpenOptions;
use std::io;
use std::os::fd::AsRawFd;
use std::ptr;
use std::slice;

fn main() -> io::Result<()> {
    let mut file_stat = unsafe { std::mem::zeroed() };
    let mut file_content: [u8; 10] = [0; 10];
    let new_data = "\nUpdated Data\n";
    let file_path = "file.txt";

    let file = OpenOptions::new().read(true).write(true).open(&file_path)?;

    unsafe {
        libc::fstat(file.as_raw_fd(), &mut file_stat);

        let mapped_memory = libc::mmap(
            ptr::null_mut(),
            file_stat.st_size as usize,
            PROT_READ + PROT_WRITE,
            MAP_SHARED,
            file.as_raw_fd(),
            0,
        );

        if mapped_memory == MAP_FAILED {
            return Err(io::Error::last_os_error());
        }

        let mapped_slice = slice::from_raw_parts(mapped_memory as *const u8, 10);
        file_content.copy_from_slice(mapped_slice);
        println!("Read: {}", String::from_utf8_lossy(&file_content));

        let new_data_bytes = new_data.as_bytes();
        let write_offset = 5;
        if file_stat.st_size as usize >= write_offset + new_data_bytes.len() {
            ptr::copy_nonoverlapping(
                new_data_bytes.as_ptr(),
                (mapped_memory as *mut u8).wrapping_add(write_offset),
                new_data_bytes.len(),
            );
            println!("Write successful at offset {} with data: {}", write_offset, new_data);
        } else {
            eprintln!("Write offset exceeds file size. Update not performed.");
        }

        libc::munmap(mapped_memory, file_stat.st_size as usize);
    }

    Ok(())
}

main()

Read: Hello Rust
Write successful at offset 5 with data: 
Updated Data



Ok(())

```sh
+---------------------------+
|                           |
|   Open file.txt           |
|                           |
+-------------+-------------+
              |
              v
+-------------+-------------+
|                           |
|   Get file information    |
|   using fstat()           |
|                           |
+-------------+-------------+
              |
              v
+-------------+-------------+
|                           |
|   Map file into memory    |
|   with read and write     |
|   permissions             |
|                           |
+-------------+-------------+
              |
              v
+-------------+-------------+
|                           |
|   Check if mapping        |
|   is successful           |
|                           |
+-------------+-------------+
              |
              v
+-------------+-------------+
|                           |
|   Read 10 bytes from the  |
|   mapped memory           |
|                           |
+-------------+-------------+
              |
              v
+-------------+-------------+
|                           |
|   Copy read data to       |
|   file_content array      |
|                           |
+-------------+-------------+
              |
              v
+-------------+-------------+
|                           |
|   Write new data to       |
|   mapped memory at        |
|   offset 5                |
|                           |
+-------------+-------------+
              |
              v
+-------------+-------------+
|                           |
|   Unmap the memory        |
|                           |
+-------------+-------------+
```

In this code snippet, we make use of the Rust standard library and incorporate certain unsafe bindings to work with the **mmap system call** and its associated operations. The utilization of the **unsafe** block is essential for managing **low-level** operations, particularly the direct copying of memory.

This code snippet illustrates the **`mmap`** system call's functionality in generating a mapped memory region. Key parameters, including the **starting address**, **size**, and **accessibility**, are explicitly defined to ensure alignment with file operations. Furthermore, common memory operations such as **reading** and **writing** to the mapped memory are executed through Rust's standard library functions, ensuring both type safety and effective memory management.

Comprehending **mmap** is crucial, particularly in the context of Dirty COW, where the exploitation depends on manipulating memory mappings to obtain unauthorized access. The Rust programming language, famous for its emphasis on safety and performance, serves as a proficient platform for navigating the complexities of low-level memory interactions while maintaining the integrity of the codebase.

---
---